## XGBoost + LGBM Ensemble by both building and state

In [1]:
import sklearn
import xgboost as xgb
from evaluation import RMSE
from evaluation import helper_func
import pandas as pd
import gc
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgbm
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
import optuna
import math
from sklearn.model_selection import cross_val_score, KFold

#### Preparing and reading data

In [2]:
data = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")
submission = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/test.csv")

In [3]:
len(submission)

9705

### Feature engineering

#### delete columns

In [4]:
reduced = pd.read_csv("/Users/charlottefelius/documents/wids2022/WIDS/data/train.csv")

# to_delete = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed",
#              "days_above_90F", "days_above_110F", "Year_Factor"]

to_delete_less = ["days_with_fog", "direction_peak_wind_speed", "direction_max_wind_speed", "max_wind_speed"]
# to_delete_less = ["building_class"]

def delete_cols(dataframe, columns):
    for colname in columns:
        del dataframe[colname]

delete_cols(reduced, to_delete_less)
delete_cols(submission, to_delete_less)

# # collect garbage
# gc.collect()


#### Temperature difference

In [5]:
min_temp = [string for string in reduced.columns if 'min_temp' in string]
max_temp = [string for string in reduced.columns if 'max_temp' in string]

In [6]:
below_ex = [string for string in reduced.columns if 'days_below_30F' in string]
above_ex = [string for string in reduced.columns if 'days_above_80F' in string]

In [7]:
# # Per month, do max - min

for min_, max_ in zip(min_temp, max_temp):
    name = min_.split("min")[0] + "diff"
    reduced[name] = reduced[max_] - reduced[min_]
    
for min_, max_ in zip(min_temp, max_temp):
    name = min_.split("min")[0] + "diff"
    submission[name] = submission[max_] - submission[min_]

In [8]:
# days_below_10f, days above_100f

for min_, max_ in zip(below_ex, above_ex):
    name = "extreme_temp"
    reduced[name] = reduced[max_] - reduced[min_]
    
for min_, max_ in zip(below_ex, above_ex):
    name = "extreme_temp"
    submission[name] = submission[max_] - submission[min_]

In [9]:
# # temp diff aug - january

# reduced["climate"] = (reduced["august_avg_temp"] - reduced["january_avg_temp"]) * reduced[['january_diff', 'february_diff',
#                                                                                            'march_diff', 'april_diff', 'may_diff', 'june_diff',
#                                                                                           'july_diff', 'august_diff', 'september_diff',
#                                                                                           'october_diff', 'november_diff', 'december_diff']].mean(axis=1)


# submission["climate"] = (submission["august_avg_temp"] - submission["january_avg_temp"]) * submission[['january_diff', 'february_diff',
#                                                                                            'march_diff', 'april_diff', 'may_diff', 'june_diff',
#                                                                                           'july_diff', 'august_diff', 'september_diff',
#                                                                                           'october_diff', 'november_diff', 'december_diff']].mean(axis=1)



In [10]:
# delete_cols(reduced, min_temp)
# delete_cols(reduced, max_temp)
# delete_cols(submission, min_temp)
# delete_cols(submission, max_temp)

#### Impute variables 

In [11]:
# take average per facility type for star_rating

# red = reduced.groupby("facility_type").mean()["energy_star_rating"]
# sub = submission.groupby("facility_type").mean()["energy_star_rating"]

# fill_dict = {}

# for i, j in zip(red.items(), sub.items()): 
#     avg = (i[1] + j[1]) / 2
#     if math.isnan(avg) == False:
#         fill_dict[i[0]] = avg
    
# nans = []

# for i, j in zip(red.items(), sub.items()): 
#     avg = (i[1] + j[1]) / 2
#     if math.isnan(avg) == False:
#             continue
#     nans.append(i[0])

# for key in nans:
#     fill_dict[key] = 10
    
# for i,j in reduced[reduced['energy_star_rating'].isna()]["facility_type"].items():
#     reduced.loc[i,'energy_star_rating'] = fill_dict[j]
    
# for i,j in submission[submission['energy_star_rating'].isna()]["facility_type"].items():
#     submission.loc[i,'energy_star_rating'] = fill_dict[j]

In [12]:
# impute with mean, ensures highest correlation

# reduced['energy_star_rating'] = reduced['energy_star_rating'].fillna(56.0)
# submission['energy_star_rating'] = submission['energy_star_rating'].fillna(56.0)
reduced['year_built'] = reduced['year_built'].fillna(reduced['year_built'].median())
submission['year_built'] = submission['year_built'].fillna(submission['year_built'].median())

#### deal with state 6

In [13]:
# Methods

def calc_vec(state, dataset):
    vec = []
    
    temp = dataset[dataset["State_Factor"] == state]
    
    for t in alltemps:
            vec.append(temp[t].mean())
    
    return np.asarray(vec)

def euclidian_distance_extensive(original, vectors, dataset):
    
    minval = 999
    name = "Null"
    
    for s in allstates:
        temp = dataset[dataset["State_Factor"] == s]
        
        orig = []
        diff = []
        
        for t in alltemps:
            orig.append(original[t].mean())
            diff.append(temp[t].mean())

        o = np.asarray(orig)
        d = np.asarray(diff)

        dist = np.linalg.norm(o-d)
        print(f'{s} has value {dist}')
        
        if dist < minval:
            minval = dist
            name = s
    
    print()
    print(f"Final value: ({name, minval})")
    return (name, minval)

def get_vector(row):
    
    vec = []
    for col in alltemps:
        vec.append(row[col])
    
    return np.asarray(vec)

def get_distance(row, distances):
    
    min_dist = 999
    state = "Null"
    
    vec = get_vector(row)

    for key, value in distances.items():
        
        dist = np.linalg.norm(value-vec)
        
#         print(f"LOG: DIST={dist}, STATE={key}")
        
        if dist < min_dist:
            min_dist = dist
            state = key
        
#     print(f"LOG: MIN_DIST={min_dist}, STATE={state}")
    return min_dist, state

In [14]:
state6 = reduced[reduced["State_Factor"] == "State_6"]
ids = []
keys = []
alltemps = [string for string in data.columns if 'temp' in string]
allstates = list(reduced["State_Factor"].unique())
del allstates[3]

state_values = dict()

for i in allstates:
    state_values[i] = calc_vec(i, data)

for i in range(len(state6)):
    row = state6.iloc[i]
    ids.append(row['id'])
    dist, k = get_distance(row, state_values)
    keys.append(k)
    
mergedf = pd.DataFrame()
mergedf["id"] = ids
mergedf["second_state"] = keys

reduced = reduced.merge(mergedf, how='left', on='id')

#### Onehot encoding

In [15]:
# onehot = ["facility_type", "State_Factor", "building_class"]
onehot = ["State_Factor", "building_class"]
"building_class"

def onehotter(dataframe, to_onehot):
    
    for i in to_onehot:
        ohe_df = pd.get_dummies(dataframe[i], prefix=i)

        # concat with original data
            
        dataframe = pd.concat([dataframe, ohe_df], axis=1).drop([i], axis=1)
        
    return dataframe

In [16]:
submission = onehotter(submission, onehot)
reduced = onehotter(reduced, onehot)

#### Group facility

In [17]:
groups_facility_ = ["Mixed_Use_", "Commercial_", "Data_", "Education_", "Food_", "Health_", "Lodging_", "Warehouse_",
                   "Service_", "Retail_", "Public_Assembly_", "Public_Safety_", "Office_", "_Unit_Building"]

for name in groups_facility_:
    newname = name + "new"
    reduced["facility_type"] = reduced.facility_type.str.replace(name, newname)
    reduced["facility_type"] = reduced["facility_type"].str.split('new').str[0]
    submission["facility_type"] = submission.facility_type.str.replace(name, newname)
    submission["facility_type"] = submission["facility_type"].str.split('new').str[0]

In [18]:
# # groups_facility_ = ["Mixed_Use_", "Office_",  "Lodging_", "Service_", "Retail_", "Public_Safety_", 
# #                      "_Unit_Building"]

# for name in groups_facility_:
#     newname = name + "new"
#     reduced["facility_type"] = reduced.facility_type.str.replace(name, newname)
#     reduced["facility_type"] = reduced["facility_type"].str.split('new').str[0]
#     submission["facility_type"] = submission.facility_type.str.replace(name, newname)
#     submission["facility_type"] = submission["facility_type"].str.split('new').str[0]

In [19]:
# find all columns with this regex, replace value by one of the new names

# for name in groups_facility_:
#     newname = name + "new"
#     reduced["facility_type"] = reduced.facility_type.str.replace(name, newname)
#     reduced["facility_type"] = reduced["facility_type"].str.split('new').str[0]

In [20]:
# also onehot encode

onehot = ["facility_type"]
reduced = onehotter(reduced, onehot)
submission = onehotter(submission, onehot)

#### Evaluation

In [21]:
def RMSE(original, predicted):
    
    aggregate = 0
    
    for orig, pred in zip(original, predicted):
        aggregate += (orig - pred)**2
    
    RMSE_ = math.sqrt(1/len(original) * aggregate)
        
    print(f'RMSE: {RMSE_}')
    
    return RMSE_

## Ensemble model

#### Divide per state & building type

In [22]:
# train model per state_factor
# train model per building_type
# train catboost per building_type!

red_commercial = reduced[reduced['building_class_Commercial'] == 1]
red_residential = reduced[reduced['building_class_Residential'] == 1]
del red_commercial['building_class_Commercial']
del red_residential['building_class_Residential']

sub_commercial = submission[submission['building_class_Commercial'] == 1]
sub_residential = submission[submission['building_class_Residential'] == 1]
del sub_commercial['building_class_Commercial']
del sub_residential['building_class_Residential']

# reduced commercial
r_com_sf1 = red_commercial[(red_commercial["State_Factor_State_1"] == 1) | (red_commercial["State_Factor_State_11"] == 1)]
r_com_sf4 = red_commercial[(red_commercial["State_Factor_State_4"] == 1) | (red_commercial["second_state"] ==  "State_4") 
                           | (red_commercial["State_Factor_State_8"] == 1) | (red_commercial["second_state"] ==  "State_8")
                           | (red_commercial["State_Factor_State_10"] == 1) | (red_commercial["second_state"] ==  "State_10")]
r_com_sf2 = red_commercial[(red_commercial["State_Factor_State_2"] == 1) | (red_commercial["second_state"] ==  "State_2")]

# submission commercial
s_com_sf1 = sub_commercial[(sub_commercial["State_Factor_State_1"] == 1) |
                          (sub_commercial["State_Factor_State_11"] == 1)]
s_com_sf2 = sub_commercial[sub_commercial["State_Factor_State_2"] == 1 ]
s_com_sf4 = sub_commercial[(sub_commercial["State_Factor_State_4"] == 1) | 
                           (sub_commercial["State_Factor_State_8"] == 1) |
                          (sub_commercial["State_Factor_State_10"] == 1)]

# reduced residential
r_res_sf1 = red_residential[(red_residential["State_Factor_State_1"] == 1) | (red_residential["State_Factor_State_11"] == 1)]
r_res_sf2 = red_residential[(red_residential["State_Factor_State_2"] == 1) | (red_residential["second_state"] ==  "State_2")]
r_res_sf4 = red_residential[(red_residential["State_Factor_State_4"] == 1) | (red_residential["second_state"] ==  "State_4") 
                            | (red_residential["State_Factor_State_8"] == 1) | (red_residential["second_state"] ==  "State_8")
                           | (red_residential["State_Factor_State_10"] == 1) | (red_residential["second_state"] ==  "State_10")]


# submission residential
s_res_sf1 = sub_residential[(sub_residential["State_Factor_State_1"] == 1) |
                           (sub_residential["State_Factor_State_11"] == 1)]
s_res_sf2 = sub_residential[sub_residential["State_Factor_State_2"] == 1 ]
s_res_sf4 = sub_residential[(sub_residential["State_Factor_State_4"] == 1) |
                           (sub_residential["State_Factor_State_10"] == 1) |
                            (sub_residential["State_Factor_State_8"] == 1) ]

# delete unnecessary columns

r_com_sf1 = r_com_sf1[r_com_sf1.columns.drop(list(r_com_sf1.filter(regex='State_Factor_State_')))]
r_com_sf2 = r_com_sf2[r_com_sf2.columns.drop(list(r_com_sf2.filter(regex='State_Factor_State_')))]
r_com_sf4 = r_com_sf4[r_com_sf4.columns.drop(list(r_com_sf4.filter(regex='State_Factor_State_')))]
r_res_sf1 = r_res_sf1[r_res_sf1.columns.drop(list(r_res_sf1.filter(regex='State_Factor_State_')))]
r_res_sf2 = r_res_sf2[r_res_sf2.columns.drop(list(r_res_sf2.filter(regex='State_Factor_State_')))]
r_res_sf4 = r_res_sf4[r_res_sf4.columns.drop(list(r_res_sf4.filter(regex='State_Factor_State_')))]

s_com_sf1 = s_com_sf1[s_com_sf1.columns.drop(list(s_com_sf1.filter(regex='State_Factor_State_')))]
s_com_sf2 = s_com_sf2[s_com_sf2.columns.drop(list(s_com_sf2.filter(regex='State_Factor_State_')))]
s_com_sf4 = s_com_sf4[s_com_sf4.columns.drop(list(s_com_sf4.filter(regex='State_Factor_State_')))]
s_res_sf1 = s_res_sf1[s_res_sf1.columns.drop(list(s_res_sf1.filter(regex='State_Factor_State_')))]
s_res_sf2 = s_res_sf2[s_res_sf2.columns.drop(list(s_res_sf2.filter(regex='State_Factor_State_')))]
s_res_sf4 = s_res_sf4[s_res_sf4.columns.drop(list(s_res_sf4.filter(regex='State_Factor_State_')))]

# weather etc is state specific? So maybe also different model for weather and then for the other factors

## Run Model

In [23]:
all_data = [r_com_sf1, r_com_sf2, r_com_sf4, r_res_sf1,
           r_res_sf2, r_res_sf4]

all_data_sub = [s_com_sf1, s_com_sf2, s_com_sf4, s_res_sf1,
           s_res_sf2, s_res_sf4]

In [24]:
def submit(data_train, data_test):
    
    def get_pred(model_name):

        X = model_name
        ids = X["id"]
        X.pop("id")

        return X, ids

    def get_X_y(model_name):

        X = model_name
        y = X['site_eui']
        y_id = X['id']
        X.pop('site_eui')
        X.pop('id')

        return X, y, y_id

    def runmodel_predict(X, y, testdata, model):
        model.fit(X, y)
        preds = model.predict(testdata)
        return preds

    def run_types_pred(data_train, data_test):

        ids = []
        resultlist_xgb = []
        resultlist_lgbm = []
        counter = 0

        for train, test in zip(data_train, data_test):
            counter += 1
            
            print(f"Training model {counter}/{len(data_train)}")
            
            if (len(train) < 2) | (len(test) == 0):
                print(f"WARNING: Empty training or test set, skipping {len(test)} values")
                continue
                
                
            X, y, y_idx = get_X_y(train)
            X.pop("second_state")
            testdata, y_id = get_pred(test)
            ids.append(y_id)
            
            # XGBoost
            print("XGBoost")
            data_dmatrix = xgb.DMatrix(data=X,label=y)
            model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.2,
                    max_depth = 11, alpha = 0.2, n_estimators = 1000)

            predicted = runmodel_predict(X, y, testdata, model)
            resultlist_xgb.append(predicted)

            # LightGBM

            print("LightGBM")
            model = lgbm.LGBMRegressor(max_depth=10, learning_rate=0.2, n_estimators=2600)
            predicted = runmodel_predict(X, y, testdata, model)
            resultlist_lgbm.append(predicted)

        return resultlist_xgb, resultlist_lgbm, ids
    
    return run_types_pred(data_train, data_test)

In [25]:
to_concat_xgb_t, to_concat_lgbm_t, ids = submit(all_data, all_data_sub)

Training model 1/6
XGBoost
LightGBM
Training model 2/6
XGBoost
LightGBM
Training model 3/6
XGBoost
LightGBM
Training model 4/6
XGBoost
LightGBM
Training model 5/6
XGBoost
LightGBM
Training model 6/6
XGBoost
LightGBM


In [26]:
ids2 = [i for si in ids for i in si]

In [27]:
# ids2 heeft id's

ids2 = [i for si in ids for i in si]
xgb = []
lgbm = []

for i in to_concat_xgb_t:
    xgb.append(i)
    
for i in to_concat_lgbm_t:
    lgbm.append(i)
    

xgb = [item for sublist in xgb for item in sublist]
lgbm = [item for sublist in lgbm for item in sublist]

In [28]:
len(xgb), len(lgbm), len(ids2)

(9705, 9705, 9705)

In [29]:
avg_predicted = []

for i, j in zip(xgb, lgbm):
    avg_predicted.append((i*2+j)/3)
    
avg_predicted2 = []

for i, j in zip(xgb, lgbm):
    avg_predicted2.append((i+j)/2)

In [30]:
print(xgb[:5])
print(lgbm[:5])
print(avg_predicted[:5])
print(avg_predicted2[:5])
print(ids2[:5])

[197.58763, 236.48914, 176.75452, 232.97636, 231.80457]
[177.48198458941116, 174.81983297888357, 185.42676994637088, 219.8602316857091, 195.07377835096472]
[190.88574901352766, 215.93270148775287, 179.64526771649864, 228.60431998573117, 219.56096973677992]
[187.53480790749853, 205.65448436053555, 181.0906432739667, 226.41829791072564, 213.43917189032612]
[75757, 75758, 75759, 75760, 75761]


In [31]:
raise NotImplementedError()

NotImplementedError: 

In [32]:
result2 = pd.DataFrame()
result2["id"] = ids2
result2["site_eui"] = avg_predicted2
result2 = result2.sort_values(by=['id'])
result2.head()

,id,site_eui
0,75757,187.534808
1,75758,205.654484
2,75759,181.090643
3,75760,226.418298
4,75761,213.439172


In [33]:
result = pd.DataFrame()
result["id"] = ids2
result["site_eui"] = avg_predicted
result = result.sort_values(by=['id'])
result.head()

,id,site_eui
0,75757,190.885749
1,75758,215.932701
2,75759,179.645268
3,75760,228.604320
4,75761,219.560970


In [34]:
result_xgb = pd.DataFrame()
result_xgb["id"] = ids2
result_xgb["site_eui"] = xgb
result_xgb = result_xgb.sort_values(by=['id'])
result_xgb.head()

,id,site_eui
0,75757,197.587631
1,75758,236.489136
2,75759,176.754517
3,75760,232.976364
4,75761,231.804565


In [35]:
result_lgbm = pd.DataFrame()
result_lgbm["id"] = ids2
result_lgbm["site_eui"] = lgbm
result_lgbm = result_lgbm.sort_values(by=['id'])
result_lgbm.head()

,id,site_eui
0,75757,177.481985
1,75758,174.819833
2,75759,185.426770
3,75760,219.860232
4,75761,195.073778


In [36]:
result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_mix1.csv', index=False, header=True)

In [ ]:
result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_mix1.csv', index=False, header=True)result_lgbm.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_lgbm2.csv', index=False, header=True)

#### Write to CSV

In [37]:
# Write to CSV file

result.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_mix1111.csv', index=False, header=True)
result2.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_mix2222.csv', index=False, header=True)
result_xgb.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_xgb222.csv', index=False, header=True)
result_lgbm.to_csv('/Users/charlottefelius/documents/WIDS2022/WIDS/results/xgb_lgbm_state_build_lgbm222.csv', index=False, header=True)

In [ ]:
stop